In [20]:
import pandas as pd
import json

In [21]:
c01t01=pd.read_csv("Client Files/c01t01.csv")
c01t02=pd.read_csv("Client Files/c01t02.csv")
c01t03=pd.read_csv("Client Files/c01t03.csv")
c02t01=pd.read_csv("Client Files/c02t01.csv")
c02t02=pd.read_csv("Client Files/c02t02.csv")
c02t03=pd.read_csv("Client Files/c02t03.csv")

In [22]:
with open("Client Json/client01.json", "r") as f:
    c01_map = json.load(f)

with open("Client Json/client02.json", "r") as f:
    c02_map = json.load(f)

In [25]:
dd=pd.read_csv("Client Files/data_dictionary.csv")
dd

,destination_file,destination_column
0,ft01,name
1,ft01,age
2,ft01,phone_number
3,ft02,order_id
4,ft02,order_date
5,ft02,cost
6,ft03,namefather
7,ft03,namemother
8,ft03,namebrother


In [ ]:

clients = {
    "c01": {
        "map": c01_map,
        "tables": {
            "table01": c01t01,
            "table02": c01t02,
            "table03": c01t03
        }
    },
    "c02": {
        "map": c02_map,
        "tables": {
            "table01": c02t01,
            "table02": c02t02,
            "table03": c02t03
        }
    }
}
clients = {
    "c01": {
        "map": c01_map,
        "tables": {
            "table01": c01t01,
            "table02": c01t02,
            "table03": c01t03
        }
    },
    "c02": {
        "map": c02_map,
        "tables": {
            "table01": c02t01,
            "table02": c02t02,
            "table03": c02t03
        }
    }
}

'''

clients = {
    "c01": {
        "map": c01_map,
        "tables": {
            "table01": c01t01,
            "table02": c01t02
        }
    }
}
'''

'\n\nclients = {\n    "c01": {\n        "map": c01_map,\n        "tables": {\n            "table01": c01t01,\n            "table02": c01t02\n        }\n    }\n}\n'

In [27]:
# Build output tables from dd automatically
output_tables = {}
for file in dd["destination_file"].unique():
    cols = dd[dd["destination_file"] == file]["destination_column"].unique().tolist()
    output_tables[file] = pd.DataFrame(columns=cols + ["client"])


In [28]:
def map_and_append(source_df, mapping, table_name, output_df, client_name):
    temp = pd.DataFrame()
    for dest_col, src_col in mapping[table_name].items():
        temp[dest_col] = source_df[src_col]

    temp["client"] = client_name
    output_df = pd.concat([output_df, temp], ignore_index=True)
    return output_df


def process_client(client_name, client_map, client_tables, output_tables):
    for table_name, source_df in client_tables.items():

        # Decide destination using first destination column
        first_dest_col = list(client_map[table_name].keys())[0]

        # Find correct output table by checking which FT contains this column
        for ft_name, ft_df in output_tables.items():
            if first_dest_col in ft_df.columns:
                output_tables[ft_name] = map_and_append(
                    source_df, client_map, table_name, ft_df, client_name
                )
                break

    return output_tables


In [29]:
for client, info in clients.items():
    output_tables = process_client(
        client,
        info["map"],
        info["tables"],
        output_tables
    )


In [32]:
for ft_name, df in output_tables.items():
    df.to_csv(f"Output Files/{ft_name}.csv", index=False)
